## Step 1 - Importing the Dataset

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bwandowando/spotify-songs-with-attributes-and-lyrics")

print("Path to dataset files:", path)

/Users/macbookpro/machine-learning-kuliah/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/macbookpro/.cache/kagglehub/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/versions/19


## Step 2 - Importing the Libraries

In [2]:
import pandas as pd
import numpy as np
import time
import faiss
from annoy import AnnoyIndex
import hnswlib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

## Step 3 - Loading and Checking the Datasets

In [3]:
import os

data_path = "/Users/macbookpro/.cache/kagglehub/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/versions/19"
print(os.listdir(data_path))
df = pd.read_csv(os.path.join(data_path, "songs_with_attributes_and_lyrics.csv"))

['songs_with_attributes_and_lyrics.csv', 'songs_with_lyrics_and_timestamps.csv']


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955320 entries, 0 to 955319
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                955320 non-null  object 
 1   name              955309 non-null  object 
 2   album_name        385557 non-null  object 
 3   artists           955318 non-null  object 
 4   danceability      955320 non-null  float64
 5   energy            955320 non-null  float64
 6   key               955320 non-null  object 
 7   loudness          955320 non-null  float64
 8   mode              955320 non-null  object 
 9   speechiness       955320 non-null  float64
 10  acousticness      955320 non-null  float64
 11  instrumentalness  955320 non-null  float64
 12  liveness          955320 non-null  float64
 13  valence           955320 non-null  float64
 14  tempo             955320 non-null  float64
 15  duration_ms       955320 non-null  float64
 16  lyrics            95

In [5]:
df.describe()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,955320.000000,955320.000000,955320.000000,955320.000000,955320.000000,955320.000000,955320.000000,955320.000000,955320.000000,9.553200e+05
mean,0.550710,0.652442,-7.833732,0.083638,0.282962,0.081875,0.220190,0.488119,122.226093,2.341441e+05
std,0.169784,0.238824,3.792018,0.092929,0.311800,0.212789,0.195938,0.251468,29.536303,9.068368e+04
min,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.586000e+03
25%,0.436000,0.482000,-9.750000,0.034500,0.011900,0.000000,0.098900,0.282000,99.021000,1.849330e+05
50%,0.558000,0.687000,-7.041000,0.047800,0.142000,0.000039,0.137000,0.477000,120.661000,2.213070e+05
75%,0.675000,0.857000,-5.148000,0.087625,0.518000,0.008660,0.285000,0.690000,140.094000,2.656400e+05
max,0.993000,1.000000,4.882000,0.966000,0.996000,1.000000,1.000000,1.000000,246.130000,5.764624e+06


In [6]:
df

,id,name,album_name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,lyrics
0,0Prct5TDjAnEgIqbxcldY9,!,UNDEN!ABLE,['HELLYEAH'],0.415,0.6050,7,-11.157,1,0.0575,0.00116,0.838000,0.4710,0.1930,100.059,79500.0,"He said he came from Jamaica,\n he owned a cou..."
1,2ASl4wirkeYm3OWZxXKYuq,!!,NaN,Yxngxr1,0.788,0.6480,7,-9.135,0,0.3150,0.90000,0.000000,0.1760,0.2870,79.998,114000.0,"Fucked a bitch, now she running with my kids\n..."
2,69lcggVPmOr9cvPx9kLiiN,!!! - Interlude,Where I Belong EP,['Glowie'],0.000,0.0354,7,-20.151,0,0.0000,0.90800,0.000000,0.4790,0.0000,0.000,11413.0,"Oh, my God, I'm going crazy\n"
3,4U7dlZjg1s9pjdppqZy0fm,!!De Repente!!,Un Palo Al Agua (20 Grandes Canciones),['Rosendo'],0.657,0.8820,5,-6.340,1,0.0385,0.00740,0.000013,0.0474,0.9390,123.588,198173.0,Continuamente se extraña la gente si no puede ...
4,4v1IBp3Y3rpkWmWzIlkYju,!!De Repente!!,Fuera De Lugar,['Rosendo'],0.659,0.8930,5,-8.531,1,0.0411,0.09220,0.000019,0.0534,0.9510,123.600,199827.0,Continuamente se extraña la gente si no puede ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955315,4zMgP1HJazJJdEf6AiG8M6,NaN,NaN,['SuperKek'],0.456,0.4820,8.0,-11.199,1.0,0.0504,0.02970,0.000000,0.1110,0.0352,71.455,281962.0,And all I am is a man\n I want the world in my...
955316,5N0MQFuudsDIQpapNI5MHM,NaN,NaN,['Prasewon'],0.543,0.2900,2.0,-14.526,0.0,0.1580,0.66700,0.015100,0.1470,0.3640,172.118,87980.0,"I think I, I think I finally\n Found a way to ..."
955317,5R8xbq4SXB5Cc62Lu7cW4y,NaN,NaN,['SoulkaOuter'],0.696,0.4440,10.0,-12.894,0.0,0.0593,0.59300,0.000740,0.2130,0.3070,105.953,129370.0,Tak dayte patsanam poschitat' poteri\n Summy n...
955318,5cjecvX0CmC9gK0Laf5EMQ,NaN,NaN,NaN,0.678,0.6590,11,-5.364,0,0.3190,0.05340,0.000000,0.5530,0.1910,146.153,202235.0,"Ave Maria, Ave Maria\n ♪\n Ich bin in der Beto..."


## Step 4 - Slicing and Scaling the Features

In [7]:
features = [
    "danceability",
    "energy",
    "loudness",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
]
X = df[features].values

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors


In [ ]:
# =============================================================================
# 1. EXACT NEAREST NEIGHBORS (Brute Force with scikit-learn)
# =============================================================================
print("=" * 80)
print("1. EXACT NEAREST NEIGHBORS (Brute Force)")
print("=" * 80)

start_time = time.time()
nn_exact = NearestNeighbors(n_neighbors=k, algorithm="brute", metric="euclidean")
nn_exact.fit(X_scaled)
build_time_exact = time.time() - start_time

# Query with a random sample
query_idx = np.random.randint(0, len(X_scaled))
query_vector = X_scaled[query_idx : query_idx + 1]

start_time = time.time()
distances_exact, indices_exact = nn_exact.kneighbors(query_vector)
query_time_exact = time.time() - start_time

print(f"Build time: {build_time_exact:.6f} seconds")
print(f"Query time: {query_time_exact:.6f} seconds")
print(f"Nearest neighbors indices: {indices_exact[0]}")
print(f"Distances: {distances_exact[0]}")
print()


1. EXACT NEAREST NEIGHBORS (Brute Force)
Build time: 0.029047 seconds
Query time: 0.628587 seconds
Nearest neighbors indices: [935436 624765 624764  69115 351328 419103 691078 712468 780391 600442]
Distances: [0.         0.20252353 0.21848341 0.21925062 0.24176131 0.27594526
 0.29302971 0.29445259 0.30416589 0.31137611]



: 

In [ ]:
# =============================================================================
# 2. ANNOY (Approximate Nearest Neighbors Oh Yeah)
# =============================================================================
print("=" * 80)
print("2. ANNOY (Approximate Nearest Neighbors)")
print("=" * 80)

n_features = X_scaled.shape[1]
n_trees = 10  # More trees = better accuracy but slower

start_time = time.time()
annoy_index = AnnoyIndex(n_features, "euclidean")
for i, vector in enumerate(X_scaled):
    annoy_index.add_item(i, vector)
annoy_index.build(n_trees)
build_time_annoy = time.time() - start_time

start_time = time.time()
indices_annoy = annoy_index.get_nns_by_vector(
    X_scaled[query_idx], k, include_distances=False
)
distances_annoy = [annoy_index.get_distance(query_idx, idx) for idx in indices_annoy]
query_time_annoy = time.time() - start_time

print(f"Build time: {build_time_annoy:.6f} seconds")
print(f"Query time: {query_time_annoy:.6f} seconds")
print(f"Nearest neighbors indices: {indices_annoy}")
print(f"Distances: {distances_annoy}")
print()


2. ANNOY (Approximate Nearest Neighbors)


In [ ]:
# =============================================================================
# 3. FAISS (Facebook AI Similarity Search)
# =============================================================================
print("=" * 80)
print("3. FAISS (Facebook AI Similarity Search)")
print("=" * 80)

# Convert to float32 (FAISS requirement)
X_scaled_float32 = X_scaled.astype("float32")
query_vector_float32 = X_scaled[query_idx : query_idx + 1].astype("float32")

start_time = time.time()
faiss_index = faiss.IndexFlatL2(n_features)  # L2 (Euclidean) distance
faiss_index.add(X_scaled_float32)
build_time_faiss = time.time() - start_time

start_time = time.time()
distances_faiss, indices_faiss = faiss_index.search(query_vector_float32, k)
query_time_faiss = time.time() - start_time

print(f"Build time: {build_time_faiss:.6f} seconds")
print(f"Query time: {query_time_faiss:.6f} seconds")
print(f"Nearest neighbors indices: {indices_faiss[0]}")
print(f"Distances (L2): {distances_faiss[0]}")
print()


In [ ]:
# =============================================================================
# 4. HNSW (Hierarchical Navigable Small World)
# =============================================================================
print("=" * 80)
print("4. HNSW (Hierarchical Navigable Small World)")
print("=" * 80)

# Initialize HNSW index
start_time = time.time()
hnsw_index = hnswlib.Index(space="l2", dim=n_features)
hnsw_index.init_index(max_elements=len(X_scaled), ef_construction=200, M=16)
hnsw_index.add_items(X_scaled, np.arange(len(X_scaled)))
hnsw_index.set_ef(50)  # ef should be >= k
build_time_hnsw = time.time() - start_time

start_time = time.time()
indices_hnsw, distances_hnsw = hnsw_index.knn_query(
    X_scaled[query_idx : query_idx + 1], k=k
)
query_time_hnsw = time.time() - start_time

print(f"Build time: {build_time_hnsw:.6f} seconds")
print(f"Query time: {query_time_hnsw:.6f} seconds")
print(f"Nearest neighbors indices: {indices_hnsw[0]}")
print(f"Distances (L2): {distances_hnsw[0]}")
print()


In [ ]:
# =============================================================================
# PERFORMANCE COMPARISON
# =============================================================================
print("=" * 80)
print("PERFORMANCE COMPARISON SUMMARY")
print("=" * 80)

results_df = pd.DataFrame({
    "Method": ["Exact NN", "Annoy", "FAISS", "HNSW"],
    "Build Time (s)": [
        build_time_exact,
        build_time_annoy,
        build_time_faiss,
        build_time_hnsw,
    ],
    "Query Time (s)": [
        query_time_exact,
        query_time_annoy,
        query_time_faiss,
        query_time_hnsw,
    ],
})

print(results_df.to_string(index=False))
print()

# Speedup comparison
print("Speedup compared to Exact NN:")
print(f"Annoy Query Speedup: {query_time_exact / query_time_annoy:.2f}x")
print(f"FAISS Query Speedup: {query_time_exact / query_time_faiss:.2f}x")
print(f"HNSW Query Speedup: {query_time_exact / query_time_hnsw:.2f}x")
print()


In [ ]:
# =============================================================================
# ACCURACY COMPARISON (Recall@k)
# =============================================================================
print("=" * 80)
print("ACCURACY COMPARISON (Recall@10)")
print("=" * 80)

# Calculate recall: how many of the approximate results match exact results
exact_set = set(indices_exact[0])
annoy_set = set(indices_annoy)
faiss_set = set(indices_faiss[0])
hnsw_set = set(indices_hnsw[0])

recall_annoy = len(exact_set.intersection(annoy_set)) / k
recall_faiss = len(exact_set.intersection(faiss_set)) / k
recall_hnsw = len(exact_set.intersection(hnsw_set)) / k

print(f"Annoy Recall@{k}: {recall_annoy:.2%}")
print(f"FAISS Recall@{k}: {recall_faiss:.2%}")
print(f"HNSW Recall@{k}: {recall_hnsw:.2%}")
print()


In [ ]:
# =============================================================================
# DEMONSTRATION: Find similar tracks
# =============================================================================
print("=" * 80)
print("EXAMPLE: Finding Similar Tracks")
print("=" * 80)

if "df" in globals() or "df" in locals():
    print(f"\nQuery Track (index {query_idx}):")
    print(df.iloc[query_idx][["name", "artists"] + features])
    print("\nTop 5 Most Similar Tracks (using Exact NN):")
    for i, idx in enumerate(indices_exact[0][1:6], 1):  # Skip first (itself)
        print(f"\n{i}. {df.iloc[idx]['name']} by {df.iloc[idx]['artists']}")
        print(f"   Distance: {distances_exact[0][i]:.4f}")
